# Python Study - 2주차 자료입니다. (Jaewook Oh)

## Contents
- Decorator / Closure
- Data Types
    - datetime
    - collections
    - heapq
    - copy
    - pprint

## 1. Closure

> 자신을 둘러싼 스코프(네임스페이스)의 상태값을 기억하는 함수

함수 X가 Closure이기 위한 조건

- 함수 X는 어떤 함수 Y에 중첩되어 있어야 한다.
- 함수 X에서는 어떤 함수 Y에서 사용된 변수의 처리가 이루어져야 한다.
- 어떤 함수 Y는 함수 X를 리턴하여야 한다.

In [12]:
# Closure Example - 1
def outer_func(): #1
    message = "HI" #3
    
    def inner_func(): #4
        #print(message)
        nonlocal message
        message = "HELLO"
        print(message) #6
    
    return inner_func() #5

outer_func() #2

HELLO


1. #1에서 정의한 함수 outer_func를 #2에서 호출
2. outer_func가 실행된 이후, message라는 변수에 "HI"라는 문자열을 할당 (#3)
3. #4에서 innner_func을 정의하고 #5에서 inner_func을 호출과 동시에 리턴
4. #6에서 message 변수를 참조하여 출력. 여기에서 message는 inner_func 안에서 정의되지 않았으나, inner_func 안에서 사용되고 있기 때문에 __Free Variable__ 이라고 부른다.

> Free Variable
> 코드 블럭 안에서 사용은 되었으나, 해당 블럭 안에서 정의되지 않은 변수를 의미한다.

In [1]:
# Closure Example - 2
def outer_func(): #1
    message = "HI" #3
    
    def inner_func(): #4
        print(message) #6
    
    return inner_func #5 괄호쌍을 지웠음

print(outer_func()) #2
outer_func()()

<function outer_func.<locals>.inner_func at 0x105b82200>
HI


아까와는 달리 message가 출력되지 않음을 알 수 있음.

이는 #5에서 outer_func이 리턴할 때, inner_func 함수를 실행하지 않고,
곧장 함수 오브젝트를 리턴하였기 때문임.

실제로 출력하기 위해서는 함수 오브젝트에 ()를 붙여서 call 해야 함

In [21]:
# Closure Example - 3
def outer_func(): #1
    message = "HI" #3
    
    def inner_func(): #4
        print(message) #6
    
    return inner_func #5


def test_func():
    x = 7
    return x

y = test_func

t = []

t.append(y)
print(t[0]())

print(y())



#outer_func()
# my_func = outer_func() #2 리턴값인 inner_func을 변수에 할당
##print(id)
#print(my_func)

7
7


# Scoping Rule
# Function Pointer

아까와 마찬가지로 message는 출력되지 않으나, my_func 변수에는 inner_func 함수가 할당되어 있음을 알 수 있다. (함수 오브젝트)

In [4]:
my_func()
my_func()
my_func()

HI
HI
HI


좀 전에 선언한 my_func 변수를 실행하면, 정상적으로 inner_func이 호출되는 것을 알 수 있다. 
그러나 outer_func이 #2에서 종료되었음에도 불구하고, outer_func 함수의 로컬 변수인 message 변수를 참조하고 있다는 점이 이상함을 알 수 있다.

다음의 과정을 통해 outer_func의 로컬 변수인 message가 어떻게 inner_func에서 호출될 수 있었는지를 확인한다.

In [5]:
# message 변수가 저장된 위치를 찾는 과정

print("# dir(my_func)")
print(dir(my_func))
print()
print("# type(my_func.__closure__))")
print(type(my_func.__closure__))
print()
print("# my_func.__closure__")
print(my_func.__closure__)
print()
print("# my_func.__closure__[0]")
print(my_func.__closure__[0])
print()
print("# dir(my_func.__closure__[0])")
print(dir(my_func.__closure__[0]))
print()
print("# my_func.__closure__[0].cell_contents")
print(my_func.__closure__[0].cell_contents)

# dir(my_func)
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']

# type(my_func.__closure__))
<class 'tuple'>

# my_func.__closure__
(<cell at 0x105b840d0: str object at 0x105b53fb0>,)

# my_func.__closure__[0]
<cell at 0x105b840d0: str object at 0x105b53fb0>

# dir(my_func.__closure__[0])
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__'

# 2. Decorator

- 매우 반복적으로 되풀이되는 코드는, 그보다 더 현명한 해결책이 존재한다. (메타 프로그래밍)
- 함수에 추가적인 처리(Logging, Timing 등)를 수행하는 Wrapper Layer를 넣고 싶을 때 데코레이터 함수를 정의한다.
- 즉, 이미 만들어진 기존의 코드를 수정하지 않고도, Wrapper 함수를 이용해 여러가지 기능을 추가할 수 있기 때문이다.
- 데코레이터는 필요한 결과를 기억하도록 "Memoization"을 하는데에 Decorator를 사용할 수도 있다.

In [28]:
import time # -> 별도의 네임스페이스를 사용하겠다는 의미. 실행은 되지만, 접근하기 위해서는 파일명을 앞에 붙여 주어야 함.
#from  time import * -> 위험. #-> namespace overload ()
from functools import wraps  # -> 선택적으로 가져옴.

def timethis(func):
    # 실행 시간을 보고하는 데코레이터
    @wraps(func)
    def wrapper(*args, **kwargs):
        stime = time.time()
        result = func(*args, **kwargs)
        etime = time.time()
        print(func.__name__, etime - stime)
        return result
    return wrapper

In [29]:
@timethis
def countdown(n:int):
    while n > 0:
        n -= 1

        
stime = time.time()
countdown(10)
etime = time.time()
print(etime-stime)


countdown 9.5367431640625e-07
0.00037384033203125


In [24]:
countdown(10_000)

countdown 0.0006330013275146484


In [26]:
countdown(100_000_000)

countdown 5.6774280071258545


In [27]:
countdown(10_000_000)

countdown 0.6107609272003174


In [51]:
countdown(18_000_000)

countdown 1.0056540966033936


Decorator는 입력으로 함수를 받고, 새로운 함수를 반환한다.

```
@timethis
def countdown(n):
    pass
은 

def countdown(n):
    countdown = timethis(countdown)

와 같은 별도의 단계를 수행한 것과 동일하다.
```

@staticmethod 나 @classmethod 또는 @property와 같은 내장된 데코레이터도 동일한 동작을 한다.

In [2]:
class A:
    @classmethod
    def method(cls):
        pass

In [12]:
class B:
    def method(cls):
        pass
    method = classmethod(method)

위 두 코드는 동일한 동작을 한다. 

데코레이터는 일반적으로 Call Signature나 감싸고 있는 함수의 Return을 수정하지 않는다는 점이 중요하다.
그리고 어떠한 입력 인자라도 받을 수 있도록 *args와 **kwargs를 사용한다. 
다만 데코레이터를 사용하는 경우, @wraps를 사용하지 않으면, 데코레이터 함수가 중요한 정보를 모두 상실하게 된다. 예를 들어,

In [30]:
countdown(10000)
print(countdown.__name__)
print(countdown.__annotations__)

countdown 0.0006749629974365234
wrapper
{}


### wraps를 사용하지 않은 경우에는 

In [5]:
import time
from functools import wraps

def timethis(func):
    # 실행 시간을 보고하는 데코레이터
    def wrapper(*args, **kwargs):
        stime = time.time()
        result = func(*args, **kwargs)
        etime = time.time()
        print(func.__name__, etime - stime)
        return result
    return wrapper

@timethis
def countdown(n:int):
    while n > 0:
        n -= 1

In [6]:
print(countdown.__name__)
print(countdown.__annotations__)

# 이처럼 기존 함수가 가지고 있던 정보를 상실하게 된다.
# 다만 모든 데코레이터가 @wraps를 사용하지는 않으며, @staticmethod나 @classmethod처럼 내장된 데코레이터는 이런 규칙을 따르지 않는다.
# (대신 함수의 원본을 __func__ 속성에 담고 있다.)

wrapper
{}


### 이미 적용한 데코레이터를 취소하고, 원본 함수에 접근하고 싶은 경우에는 __wrapped__ 속성을 이용한다. #TODO

In [31]:
# Decorator Example - Logger

import datetime
import time
from functools import wraps

def my_logger(func):
    import logging
    logging.basicConfig(level=logging.INFO)
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        logging.info("[Result - {}] args - {}, kwargs - {}".format(timestamp, args, kwargs))
        return func(*args, **kwargs)

    return wrapper

@my_logger
def display_info(name, age):
    time.sleep(1)
    print("Display_info({}, {}) Function".format(name, age))


display_info("John", 25)

INFO:root:[Result - 2020-06-01 20:01] args - ('John', 25), kwargs - {}


Display_info(John, 25) Function


### 복수 개의 데코레이터를 사용하는 경우, 맨 아래의 데코레이터부터 실행 된다.

In [36]:
# 매개변수를 받는 데코레이터 정의
from functools import wraps
import logging

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            #print(level, logmsg)
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example - 1
@logged(logging.DEBUG)
def add(x, y):
    return x + y

# Example - 2
@logged(logging.CRITICAL, 'example', '[SpamMessage]')
def spam():
    print("SPAM!")
    
# Example - 3
@logged(logging.ERROR, 'example3', '[ErrorMessage]')
def error():
    print("ERROR!")
    
# Example - 4 <= Error
@logged
def noneParam():
    print("NONEPARAM!")
    
print(add(1,2))
print()
spam()
print()
error()
print()
noneParam()

CRITICAL:example:[SpamMessage]
ERROR:example3:[ErrorMessage]


3

SPAM!

ERROR!



TypeError: decorate() missing 1 required positional argument: 'func'

예제 4번의 코드는 동작하지 않는다. logged 데코레이터는 적어도 하나의 인자(func)를 요구하기 때문이다. 

## 매개변수를 받는 데코레이터를 런타임 때 속성을 수정할 수 있도록 정의하기

함수를 감싸는 데코레이터를 작성하는데, 사용자가 runtime 시에 속성을 조절할 수 있도록 접근자 함수를 만들어 해결할 수 있다.

In [52]:
from functools import wraps, partial
import logging

def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(level, logmsg)
            #log.log(level, logmsg)
            return func(*args, **kwargs)
        
        # Setter Func
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel
        
        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg
        
        return wrapper
    return decorate

# Example - 1
@logged(logging.DEBUG)
def add(x, y):
    return x + y

# Example - 2
@logged(logging.CRITICAL, 'example', '[SpamMessage]')
def spam():
    print("SPAM!")
    
# Example - 3
@logged(logging.ERROR, 'example3', '[ErrorMessage]')
def error():
    print("ERROR!")
    
# Example - 4 <= Error
@logged
def noneParam():
    print("NONEPARAM!")

print(add(2,3))
print()
add.set_message("Add called")
print(add(2,3))
print()
add.set_level(logging.WARNING)
print(add(2,3))


10 add
5

10 Add called
5

30 Add called
5


In [23]:
add(2,3)

10 add


5

In [54]:
add.set_message("Add called")
add(2,3)

30 Add called


5

In [53]:
add.set_level(logging.WARNING)
add(2,3)

30 Add called


5

##### nonlocal 지시어가 사용된 이유

```
a = 10
def outer():
    n = 10
    def inner():
        print(a)
        a += 1
        print(a)
        print(n)
        n += 1
        print(n)
        x = 10
        print(x)
        x += 1
        print(x)
     
    inner()
```
위와 같은 함수가 존재한다고 가정하자.

여기에서 a는 모든 함수에서 접근할 수 있는 global 변수이다.

outer 함수에 위치한 변수 n은 outer() 함수 입장에서는 local 범위이지만, inner() 함수 입장에서는 nonlocal 범위에 들어간다.

따라서 변수 n은 inner() 함수에서 읽기만 가능하다.

그러나 nonlocal 지시어를 사용하면 local에 위치하지 않은 변수를 불러와서 쓰기 작업을 수행할 수 있게 해준다.

global 지시어도 마찬가지의 역할을 한다.


In [10]:
x = 1
print(id(x))

def a():
    #global x
    #print(id(x))
    y = x
    print(id(y))
    x = 7 
    print(id(x))

a()
print(id(x))

4349936768


UnboundLocalError: local variable 'x' referenced before assignment

In [13]:
a = 10

In [27]:
a = 10
def outer():
    n = 10
    # global a
    print(a)
    a += 1
    print(a)
    
    def inner():
        # global a
        print(a)
        a += 1
        print(a)
        # nonlocal n
        print(n)
        n += 1
        print(n)
        x = 10
        print(x)
        x += 1
        print(x)
     
    inner()
    
outer()

10
11
11
12
10
11
10
11


 ## 호출 방식이 다른 데코레이터를 일관성 있게 하나로 작성하기 (Example - 4 Error 해결하기)

In [93]:
from functools import wraps, partial
import logging

# *의 의미는 이 이후에 나오는 파라미터는 모두 keyword 파라미터임을 의미한다.
def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)
    
    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__

    @wraps(func)
    def wrapper(*args, **kwargs):
        print(level, logmsg)
        #log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

In [94]:
@logged(level=logging.DEBUG)
def add(x, y):
    return x + y

@logged
def noneParam():
    print("NONEPARAM!")

In [33]:
add(3, 5)

10 add


8

In [95]:
noneParam()

10 noneParam
NONEPARAM!


```
# A
@logged
def add(x, y):
    return x + y
```

```
# B
def add(x, y):
    return x + y
add = logged(add)
```

A와 B는 같은 코드이다.
감싸줄 함수를 logged에 첫 번째 매개변수로 넘겨준다.
func이 None으로 왔으므로, partial 함수에 의해 func 매개변수를 받지 않는 logged 데코레이터가 생성된다.
그리고 이 코드에서 logged()의 첫 번째 매개변수는 감싸줄 함수이다. 

```
# C
@logged(level=logging.CRITICAL, name='example')
def spam():
    print('SPAM')
```

```
# D
def spam():
    print('SPAM')
spam = logged(level=logging.CRITICAL, name='example')(spam)
```

C와 D는 같은 코드이다. 


### Partial 함수에 대해 짚고 넘어가자면, 단순히 기존의 함수와 구현은 동일하나, 파라미터만 미리 정해준 형태로 또 다른 함수를 생성하는 것이다.

In [35]:
from functools import partial

def power(base, exponent):
    return base ** exponent

# 제곱을 구하는 함수를 구현하는 법 - 1
def square(num):
    return num ** 2

print(square(4))
print()

# 제곱을 구하는 함수를 구현하는 법 - 2
square = partial(power, exponent=2)
print(square)
print(square(4))

16

functools.partial(<function power at 0x108276cb0>, exponent=2)
16


## 3. Datetime

시간에 대해 단위 변환(초->분 등) 이나 단위가 다른 값에 대한 계산을 하려면 datetime 모듈을 사용한다.

In [38]:
# 시간의 간격을 나타내기 위해서 timedelta 인스턴스를 사용한다.
from datetime import timedelta
a = timedelta(days=2, hours=6)
b = timedelta(hours=4.5)
c = a + b
print("c.days =", c.days, "c.seconds =", c.seconds)

c.days = 2 c.seconds = 37800


In [59]:
# 특정 날짜와 시간을 표현하려면 datetime 인스턴스를 만들고, 연산을 수행한다.
from datetime import datetime
a = datetime(2020, 5, 25)
b = datetime(2020,5, 26)
import time

print(time.mktime(a.timetuple()))
# print(a)
# print(b-a)
#print(timedelta(a).total_seconds())
# print(a.timestamp)
print(a)
print(a + timedelta(days=10))
b = datetime(2020, 12, 25)
d = b - a
print(d.days)

now = datetime.today()
print(now)
print(now + timedelta(minutes=10))

1590332400.0
2020-05-25 00:00:00
2020-06-04 00:00:00
214
2020-06-01 20:24:21.300340
2020-06-01 20:34:21.300340


In [40]:
# datetime은 윤년을 인식하여 계산을 수행한다.
a = datetime(2020, 3, 1)
b = datetime(2020, 2, 28)
a - b
print((a - b).days)
print()

c = datetime(2021, 3, 1)
d = datetime(2021, 2, 28)
print((c - d).days)

2

1


In [35]:
# 문자열 형식의 시간 데이터를 datetime 객체로 변환하고 싶을 때 사용한다.
from datetime import datetime # from {filename} import {function}
text_day = '2020-05-25'
y = datetime.strptime(text_day, '%Y-%m-%d')
z = datetime.now()
diff = z - y
print(diff)

9:44:51.074149


- 다만 strptime() 메서드는 순수 파이썬만을 사용해서 구현하였고, 시스템 설정과 같은 세세한 부분을 모두 처리하므로 예상보다 실행 속도가 느린 경우가 많다.
- 만약 코드에서 처리해야 하는 날짜가 아주 많은데, 그 날짜 형식을 정확히 알고 있다면, 해결책을 직접 구현하는 것이 속도 측면에서 훨씬 유리하다.

In [36]:
# 예를 들어, 날짜 형식이 'YYYY-MM-DD' 라는 것이 분명하다면 다음과 같이 구현할 수 있다.
from datetime import datetime
def parse_ymd(s):
    year_s, mon_s, day_s = s.split('-')
    return datetime(int(year_s), int(mon_s), int(day_s))

# Datetime 예제

## 4. Collections

파이썬의 dict, list, set, tuple을 대신할 특수 컨테이너 데이터 타입을 제공하는 모듈.
> 기존 Collections Abstract Base Classes 모듈에서 Collections.abc 모듈로 이동하였고, 이전 버전의 호환성을 위해서 아직까지는 볼 수 있는 상태이다.

> 3.3 버전부터 Deprecated 되었고, 3.10 버전에서는 아예 삭제될 예정이다.

### 4-1 ChainMap

> dictionary를 리스트의 형태로 보관하는 자료구조로, dict를 새로 생성하거나 여러번 update()를 호출하는 것보다 빠르다. 


In [12]:
dict1 = {'apple': 1, 'banana': 2}
dict2 = {'coconut': 1, 'date': 1, 'apple': 3}

import collections

combined_dict = collections.ChainMap(dict1, dict2)
reverse_combined_dict = collections.ChainMap(dict2, dict1)

for k, v in combined_dict.items():
    print(k, v) # 먼저 만나는 key가 살아남는다.
    
print()

for k, v in reverse_combined_dict.items():
    print(k, v)

coconut 1
date 1
apple 1
banana 2

apple 3
banana 2
coconut 1
date 1


### 4-2 Counter

> 컨테이너에 동일한 값의 자료가 몇 개인지 파악하는데 사용할 수 있다.
> 리턴 값은 딕셔너리 형태

In [66]:
# 어떤 단어가 주어졌을 때, 해당 단어에 포함된 각 알파벳의 개수를 세는 함수를 짠다면

def countLetters(word):
    counter = {}
    for letter in word:
        if letter not in counter:
            counter[letter] = 0
        counter[letter] += 1
    return counter

countLetters("Hello World")

{'H': 1, 'e': 1, 'l': 3, 'o': 2, ' ': 1, 'W': 1, 'r': 1, 'd': 1}

In [67]:
# Counter를 이용하면 한 줄로 줄일 수 있다.
from collections import Counter
Counter("Hello World")

Counter({'H': 1, 'e': 1, 'l': 3, 'o': 2, ' ': 1, 'W': 1, 'r': 1, 'd': 1})

In [68]:
# most_common(k) 와 같은 메서드를 제공하여, 가장 개수가 많은 k개의 데이터를 얻을 수 있음.
Counter("Hello World").most_common() # 개수가 많은 순서대로 정렬한 리스트로 반환

[('l', 3),
 ('o', 2),
 ('H', 1),
 ('e', 1),
 (' ', 1),
 ('W', 1),
 ('r', 1),
 ('d', 1)]

#### Counter("Hello World").most_common(2) # 가장 개수가 많은 2개의 데이터를 얻을 수 있음.

### 4-3 deque

> double-ended queue로, 앞, 뒤 양방향에서 데이터를 처리할 수 있는 queue형 자료구조.
느려지지 않는 queue

In [17]:
dq = collections.deque()

# append
dq.append(10)
print(dq)
print()
dq.appendleft(-1)
print(dq)
print()

# iterable한 데이터를 끝에 추가하는 메서드 extend
dq.extend('eaf')
print(dq)
print() # list의 경우, list.append('eaf')를 하게 되면 'eaf'가 곧대로 들어간다.)

deque([10])

deque([-1, 10])

deque([-1, 10, 'e', 'a', 'f'])



### 4-4 defaultdict

> 기본 딕셔너리와 비슷하지만, key 값이 주어지지 않는 경우, 미리 지정해놓은 초기값을
key로 반환해준다.

In [61]:
# 일반 dict의 경우
dict1 = {'a': 1, 'b': 2}
print(dict1)
print(dict1['c']) # KeyError

{'a': 1, 'b': 2}


KeyError: 'c'

In [61]:
import collections
# defaultdict의 경우
def default_factory():
    return 'null'

# default_factory를 넣어주지 않으면, 기본 딕셔너리처럼 KeyError Exception이 발생.
dict2 = collections.defaultdict(default_factory, a=1, b=2)
print(dict2)
print(dict2['c'])

defaultdict(<function default_factory at 0x10935d680>, {'a': 1, 'b': 2})
null


#### 기본 딕셔너리에서도 setdefault 메서드를 통해 초기값을 지정할 수 있도록 해주나, defaultdict의 default_factory가 더 간단하고 더 빠르다.

>  The list.append() operation then attaches the value to the new list. When keys are encountered again, the look-up proceeds normally (returning the list for that key) and the list.append() operation adds another value to the list. This technique is simpler and faster than an equivalent technique using dict.setdefault():

### 4-5 namedtuple

In [20]:
# Basic Example
Point = collections.namedtuple('Point', ['x', 'y'])
p = Point(11, y=22)
print(p[0]+p[1])
print()
x, y = p
print(x, y)
print()
print(p.x+p.y)
print()
print(p)

33

11 22

33

Point(x=11, y=22)


- Named tuple은 주로 csv나 sqlite3 모듈에서 반환되는 튜플에, 필드의 이름을 할당하는데 유용하다.
- C언어의 구조체처럼 사용할 수 있다.


### 4-6 OrderedDict

> 딕셔너리와 거의 비슷하나, 입력된 item의 순서를 기억하는 딕셔너리로 sorted() 
메서드를 이용하여 정렬된 딕셔너리를 구성할 수 있다.

In [39]:
dict1 = {}
dict1['aaa'] = 'A'
dict1['bbb'] = 'B'
dict1['ccc'] = 'C'
dict1['ddd'] = 'D'

dict2 = collections.OrderedDict()
dict2['aaa'] = 'A'
dict2['bbb'] = 'B'
dict2['ccc'] = 'C'
dict2['ddd'] = 'D'

dict3 = collections.OrderedDict()
dict3['ccc'] = 'C'
dict3['bbb'] = 'B'
dict3['ddd'] = 'D'
dict3['aaa'] = 'A'

print("dict1\n", dict1.keys())
print()
print("dict1 sorted\n", sorted(dict1.keys()))
print()
print("dict2\n", dict2.keys())
print()
print("dict3\n", dict3.keys())

dict1
 dict_keys(['aaa', 'bbb', 'ccc', 'ddd'])

dict1 sorted
 ['aaa', 'bbb', 'ccc', 'ddd']

dict2
 odict_keys(['aaa', 'bbb', 'ccc', 'ddd'])

dict3
 odict_keys(['ccc', 'bbb', 'ddd', 'aaa'])


### 4-7 UserDict, UserList, UserString

#### UserDict
- 딕셔너리 객체의 wrapper 역할을 수행하는 클래스
- dict보다는 UserDict를 상속해서 매핑형을 만드는 것이 쉽다. 

#### UserList와 UserString도 비슷한 역할을 한다.


In [91]:
# 삽입, 갱신, 조회할 때, 비 문자열 키를 항상 문자열로 변환하는 함수 (일반 Dict 버전)
class strKeyDict0(dict):
    
    # 존재하지 않는 키를 처리하는 메서드
    # 기본 dict 클래스에는 정의되어 있지 않으나, dict 클래스를 상속하여 정의할 수 있다.
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        # isinstance()로 검사하지 않으면, str(key)가 없는 경우, 무한 루프에 빠진다.
        return self[str(key)]
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    
    # k in d 연산을 수행하면 __contains__() 메서드가 호출된다.
    # 그러나 dict에서 상속받게 되면, dict의 기본 __contains__() 메서드는
    # 우리가 __missing__() 메서드를 호출하지 않기 때문에
    # __contains__() 메서드도 정의해주어야 한다.
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [92]:
d = strKeyDict0([('2', 'two'), ('4', 'four')])
print(dir(d))
print()
print(d['2'])
print()
print(d[4]) # integer를 넣어도 값을 잘 가져옴.
print()
#print(d[1]) # KeyError
print()
print(1 in d)
print()
print(2 in d)
print()
print(d.get('2'))
print()
print(d.get(4)) # Get을 오버라이딩 하지 않으면 None 값이 뜸.

['__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__missing__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'clear', 'copy', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']

two

four


False

True

two

four


In [69]:
# 삽입, 갱신, 조회할 때, 비 문자열 키를 항상 문자열로 변환하는 함수 (UserDict 버전)
import collections
class strKeyDict(collections.UserDict):
    
    # 존재하지 않는 키를 처리하는 메서드
    # 위처럼 dict를 상속해 처리하는 것도 가능하나, 본 예제처럼 UserDict를 상속하여
    # 정의하는 것이 훨씬 낫다.
    # 내장형에서는 아무런 문제 없이 상속할 수 있는 메서드들을 직접 오버라이드해야 하는 
    # 구현의 특성 때문에 UserDict를 상속하는 편이 낫다. 
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    
    # 저장된 키가 모두 str 형이므로, self.data를 통해 바로 조회할 수 있다.
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [80]:
d = strKeyDict([('2', 'two'), ('4', 'four')])
print(dir(d))
print()
print(d['2'])
print()
print(d[4]) # integer를 넣어도 값을 잘 가져옴.
print()
#print(d[1]) # KeyError
print()
print(1 in d)
print()
print(2 in d)
print()
print(d.get('2'))
print(d.get)

['_MutableMapping__marker', '__abstractmethods__', '__class__', '__contains__', '__copy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__missing__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__setattr__', '__setitem__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'clear', 'copy', 'data', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'setdefault', 'update', 'values']

two

four


False

True

two


## 5. heapq

> heapq 모듈은 우선 순위 큐 알고리즘이라고도 하며, heapq 알고리즘의 구현을 제공함
힙은 모든 부모 노드가 자식 노드보다 작거나 같은 값을 가지는 이진 트리이다. 
여기서 가장 흥미로운 특성은 가장 작은 요소가 항상 root인 heap[0] 라는 것

#### 정렬된 여러 시퀀스를 병합하여 순환하고 싶을 때 사용할 수 있다.
- heapq.merge()는 아이템에 순환적으로 접근하며, 제공한 시퀀스를 한꺼번에 읽지 않는다. 따라서 아주 긴 시퀀스도 별다른 무리 없이 순환할 수 있다.
- heapq.merge()에 넣는 시퀀스는 모두 정렬되어 있어야 한다. 또한 데이터가 정렬되어 있는지 확인하지 않는다. 

In [25]:
import heapq

a = [1, 4, 7, 10]
b = [2, 5, 6, 11]
for c in heapq.merge(a, b):
    print(c, end = ' ')
print()

# 정렬이 되어 있지 않은 경우에는 어떠한 경고도 없이 이상한 결과를 출력한다.
e = [7, 4, 10, 1]
f = [5, 6, 2 ,11]
for g in heapq.merge(e, f):
    print(g, end = ' ')
print()

1 2 4 5 6 7 10 11 
5 6 2 7 4 10 1 11 


#### N개의 아이템의 최대 혹은 최솟값을 찾을 때 사용한다.
- 컬렉션 내부에서 가장 크거나, 가장 작은 N개의 아이템을 찾아야 하는 경우 heapq.nlargest()와 heapq.nsmallest() 메서드를 사용할 수 있다.
- N이 컬렉션의 전체 크기보다 작다면, 이 함수를 사용하여 해결하는 것이 더 나은 성능을 제공한다.


In [23]:
import heapq

nums = [1, 8, 2, 23, 7, -4, 18, 23, 42, 37, 2]
print(heapq.nlargest(3, nums))
print()
print(heapq.nsmallest(3, nums))
print()
portfolio = [
    {'name': 'IBM', 'shares': 100, 'price': 91.1},
    {'name': 'AAPL', 'shares': 50, 'price': 543.22},
    {'name': 'FB', 'shares': 200, 'price': 21.09},
    {'name': 'HPQ', 'shares': 35, 'price': 31.75},
    {'name': 'YHOO', 'shares': 45, 'price': 16.35},
    {'name': 'ACME', 'shares': 75, 'price': 115.65}
]
cheap = heapq.nsmallest(3, portfolio, key=lambda s: s['price'])
expensive = heapq.nlargest(3, portfolio, key=lambda s: s['price'])
print(cheap)
print()
print(expensive)

[42, 37, 23]

[-4, 1, 2]

[{'name': 'YHOO', 'shares': 45, 'price': 16.35}, {'name': 'FB', 'shares': 200, 'price': 21.09}, {'name': 'HPQ', 'shares': 35, 'price': 31.75}]

[{'name': 'AAPL', 'shares': 50, 'price': 543.22}, {'name': 'ACME', 'shares': 75, 'price': 115.65}, {'name': 'IBM', 'shares': 100, 'price': 91.1}]


In [9]:
import heapq

heap = []

# 힙에 원소 추가하기
heapq.heappush(heap, 4)
heapq.heappush(heap, 1)
heapq.heappush(heap, 7)

print(heap)

# heappush()과 heappop() 메서드는 O(logN)의 시간복잡도를 갖는다.
# 이는 내부적으로 heap 형태를 유지하면서 데이터를 추가하거나 뽑기 때문이다.

#힙에서 원소 삭제하기
print(heapq.heappop(heap))
print(heap)

# 최솟값 확인하기
print(heap[0])

# 기존의 리스트를 힙으로 변환하기 O(n)
heap = [4, 1, 7, 3, 8, 5]
heapq.heapify(heap)
print(heap)

[1, 4, 7]
1
[4, 7]
4
[1, 3, 5, 4, 8, 7]


## 6. copy

### 6-1 얕은 복사

In [55]:
# list를 가지고 예를 들어보기
a = [1,2,3]
b = a[:] 

print("id(a) : ", id(a))
print("id(b) : ", id(b))

print("a == b : ", a == b)
print("a is b : ", a is b)

b[0] = 5

print("a: ", a) # b가 달라진다고 해서 a가 달라지는 것은 아님.
print("b: ", b)

id(a) :  4608896128
id(b) :  4600832160
a == b :  True
a is b :  False
a:  [1, 2, 3]
b:  [5, 2, 3]


In [59]:
# 문제는 mutable 객체 내에 mutable 객체가 있는 경우에 발생.
a = [[1,2], [3,4]]
b = a[:]

print("id(a) : ", id(a))
print("id(b) : ", id(b))

print("id(a[0]) : ", id(a[0]))
print("id(b[0]) : ", id(b[0]))

print("a is b : ", a is b)
print("a[0] is b[0] : ", a[0] is b[0]) # 겉은 다르지만, 속은 같다!

id(a) :  4586968560
id(b) :  4609258752
id(a[0]) :  4614073312
id(b[0]) :  4614073312
a is b :  False
a[0] is b[0] :  True


In [62]:
# 재할당하는 경우에는 메모리 주소가 변경되므로 문제가 없음
a[0] = [8,9]
print("a : ", a)
print("b : ", b)
print("id(a[0]) : ", id(a[0]))
print("id(b[0]) : ", id(b[0]))
print("a[0] is b[0] : ", a[0] is b[0]) # a[0]을 재할당하여 메모리 주소도 달라짐.

a :  [[8, 9], [3, 4]]
b :  [[1, 2], [3, 4]]
id(a[0]) :  4598125184
id(b[0]) :  4614073312
a[0] is b[0] :  False


In [63]:
# 그러나 a[1]의 값을 "변경"하면 문제가 발생. b[1]도 함께 변경됨
a[1].append(5)
print("a : ", a)
print("b : ", b)
print("id(a[1]) : ", id(a[1]))
print("id(b[1]) : ", id(b[1]))
print("a[1] is b[1] : ", a[1] is b[1]) # a[1]의 값을 변경하면 b[1]도 함께 변경됨.

a :  [[8, 9], [3, 4, 5]]
b :  [[1, 2], [3, 4, 5]]
id(a[1]) :  4591164064
id(b[1]) :  4591164064
a[1] is b[1] :  True


In [64]:
# copy 모듈의 얕은 복사 메서드인 copy()도 같은 상황이 발생함
import copy
a = [[1,2], [3,4]]
b = copy.copy(a)
a[1].append(5)
print("a : ", a)
print("b : ", b)

a :  [[1, 2], [3, 4, 5]]
b :  [[1, 2], [3, 4, 5]]


In [ ]:
# 이러한 상황을 해결하기 위해서 깊은 복사가 필요함

### 6-2 깊은 복사

In [65]:
a = [[1,2], [3,4]]
b = copy.deepcopy(a) # 내부의 객체까지 모두 새롭게 copy되는 방식.
a[1].append(5)
print("a : ", a)
print("b : ", b)

a :  [[1, 2], [3, 4, 5]]
b :  [[1, 2], [3, 4]]


## 7. pprint

### 7-1 리스트를 간결하게 출력해보기

In [6]:
numbers = [[1, 2, 3], [4, 5], [6, 7, 8, 9]]
print(numbers)
print()
print(*numbers)

from pprint import pprint, PrettyPrinter
print()

# 일반 출력(자동으로 줄바꿈이 이루어짐)
pprint(numbers)
print()

# 간격 설정 추가
pprint(numbers, width=20)
print()

# 들여쓰기 설정 추가
pprint(numbers, width=20, indent=4)
print()

# PrettyPrinter 객체를 생성해서 쓰기
pp = PrettyPrinter(width=20, indent=4)
pp.pprint(numbers)

[[1, 2, 3], [4, 5], [6, 7, 8, 9]]

[1, 2, 3] [4, 5] [6, 7, 8, 9]

[[1, 2, 3], [4, 5], [6, 7, 8, 9]]

[[1, 2, 3],
 [4, 5],
 [6, 7, 8, 9]]

[   [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9]]

[   [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9]]


### 7-2 딕셔너리를 간결하게 출력해보기

In [5]:
from pprint import pprint

info = {"name":'oh', "age":26, "addr":'dongjakgu'}
print(info)
print()

print(*info)
print()

print([k for k in info])
print()

print([(k, info[k]) for k in info])
print()

print(*[(k, info[k]) for k in info])
print()

# 일반 출력
pprint(info)
print()

# 간격 설정 추가
pprint(info, width=20)
print()

# 들여쓰기 설정 추가
pprint(info, width=20, indent=4)

{'name': 'oh', 'age': 26, 'addr': 'dongjakgu'}

name age addr

['name', 'age', 'addr']

[('name', 'oh'), ('age', 26), ('addr', 'dongjakgu')]

('name', 'oh') ('age', 26) ('addr', 'dongjakgu')

{'addr': 'dongjakgu', 'age': 26, 'name': 'oh'}

{'addr': 'dongjakgu',
 'age': 26,
 'name': 'oh'}

{   'addr': 'dongjakgu',
    'age': 26,
    'name': 'oh'}


## 생각해볼 문제
### n번째 피보나치 수를 구하는 코드를 데코레이터를 활용하여 구현해보자.

[go to the answer](./answer.py)

## 참고
* [Closure-1](http://schoolofweb.net/blog/posts/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%ED%81%B4%EB%A1%9C%EC%A0%80-closure/)
* [Closure-2](https://shoark7.github.io/programming/python/closure-in-python)
* [ChainMap](https://riptutorial.com/ko/python/example/30550/컬렉션--chainmap)
* [UserDict](https://books.google.co.kr/books?id=NJpIDwAAQBAJ&pg=PA124&lpg=PA124&dq=userdict&source=bl&ots=el-ThKLKWt&sig=ACfU3U05dlBwlFk2YUY73s8F78JlR0pexQ&hl=ko&sa=X&ved=2ahUKEwjAyv6X3M3pAhVVJaYKHQ3UD8M4ChDoATAGegQICBAB#v=onepage&q=userdict&f=false)
* [Python Cookbook](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwj-sbTx683pAhVNFqYKHYMxBgAQFjAAegQIAxAB&url=https%3A%2F%2Fwww.amazon.com%2FPython-Cookbook-Third-David-Beazley%2Fdp%2F1449340377&usg=AOvVaw3sPntbcfHMFU_LS4nBnlje)
* [Partial](http://www.incodom.kr/%ED%8C%8C%EC%9D%B4%EC%8D%AC/%ED%95%A8%EC%88%98)